## Dataset Analysis ##

In [3]:
from Model import Model as DataModel
from collections import Counter

dataset = []

# for filename in ["../dataset_raw/semeval2016-task6-trialdata.txt", "../dataset_raw/semeval2016-task6-trainingdata.txt"]:
#     f = open(filename, 'r')
#     f.readline() # Skip the first line which contains the title

#     for line in f.readlines():
#         items = line.strip().split('\t')
#         t = DataModel(items[0] , items[1], items[2], items[3])
#         dataset.append(t)

targets_small_hack = ['atheism', 'climate change is a real concern', 'feminist movement', 'legalization of abortion', 'hillary clinton']
targets_skip_index = [1, 6, 3, 4, 3]
targets_real =  ['Atheism', 'Climate Change is a Real Concern', 'Feminist Movement', 'Legalization of Abortion', 'Hillary Clinton']

f = open("../final.txt", 'r')
for line in f.readlines():
    items = line.strip().split('\t')
    
    for t in xrange(5):
        if items[0].startswith(targets_small_hack[t]):
            model = DataModel(None, targets_real[t], items[0][targets_skip_index[t]:], items[-1])
            dataset.append(model)        
        
targets = list(set(map(lambda model:model.target, dataset)))
stances = list(set(map(lambda model: model.stance, dataset)))

print "Stances = ", stances
print "Targets = ", targets
print "Total Tweets =", len(dataset)
for t in targets:
    print t, len([_ for x in dataset if x.target == t]), Counter([x.stance for x in dataset if x.target == t])

 Stances =  ['FAVOR', 'NONE', 'AGAINST']
Targets =  ['Atheism', 'Legalization of Abortion', 'Feminist Movement', 'Climate Change is a Real Concern', 'Hillary Clinton']
Total Tweets = 2914
Atheism 513 Counter({'AGAINST': 304, 'NONE': 117, 'FAVOR': 92})
Legalization of Abortion 653 Counter({'AGAINST': 355, 'NONE': 177, 'FAVOR': 121})
Feminist Movement 664 Counter({'AGAINST': 328, 'FAVOR': 210, 'NONE': 126})
Climate Change is a Real Concern 395 Counter({'FAVOR': 212, 'NONE': 168, 'AGAINST': 15})
Hillary Clinton 689 Counter({'AGAINST': 393, 'NONE': 178, 'FAVOR': 118})


**Reading the Stanford GloVe Twitter Embeddings learned over Twitter data**

In [4]:
import numpy as np

glove_word_vec_file = "../glove.twitter.27B/glove.twitter.27B.200d.txt"

def readGloveData(glove_word_vec_file):
    f = open(glove_word_vec_file, 'r')
    rawData = f.readlines()
    word_vec_dict = {}
    for line in rawData:
        line = line.strip().split()
        tag = line[0]
        vec = line[1:]
        word_vec_dict[tag] = np.array(vec, dtype=float)
            
    return word_vec_dict
            
word_vec_dict = readGloveData(glove_word_vec_file)

def getWordVector(word):
    if word in word_vec_dict:
        return word_vec_dict[word]
    return np.zeros_like(word_vec_dict['hi'])

def getSumVectors(tweetData):
    numNonZero = 0
    vector = np.zeros_like(word_vec_dict['hi'])
    
    for word in tweetData:
        vec = getWordVector(word)
        vector = vector + vec
        if vec.sum() != 0:
            numNonZero += 1

    if numNonZero:
        vector = vector / numNonZero

    return vector

In [5]:
from collections import Counter
from nltk import TweetTokenizer

tknzr = TweetTokenizer(strip_handles=True, preserve_case=False)

word_counter = Counter()

for tweet_model in dataset:
    tweet = tweet_model.tweet_content
    tweet = unicode(tweet, errors='ignore')
    tweet = tknzr.tokenize(tweet)

    for word in tweet:
        word_counter.update([word])
        
print "Total Words in DataSet =", len(word_counter.keys())

Total Words in DataSet = 10163


In [47]:
words_not_in_glove = Counter()

for word in word_counter.iterkeys():
    if word.lower() not in word_vec_dict:
        words_not_in_glove[word] += word_counter[word]

print "Total Words Not Present in GloVe embeddings =", len(words_not_in_glove.keys())
print "Absent Hastags #", len([_ for x in words_not_in_glove.keys() if x.startswith("#") == True])
print "Absent Other words #", len([_ for x in words_not_in_glove.keys() if x.startswith("#") == False])

absent_hashtag = Counter()
absent_other = Counter()

for x in words_not_in_glove.keys():
    if x.startswith("#") == True:
        absent_hashtag[x] += word_counter[x]
    else:
        absent_other[x] += word_counter[x]

print absent_hashtag.most_common(30)
print absent_other.most_common(30)

Total Words Not Present in GloVe embeddings = 2660
Absent Hastags # 1875
Absent Other words # 785
[(u'#semst', 2914), (u'#gamergate', 61), (u'#god', 54), (u'#lovewins', 48), (u'#wakeupamerica', 38), (u'#scotus', 35), (u'#prolifeyouth', 34), (u'#hillaryclinton', 33), (u'#feminist', 29), (u'#islam', 23), (u'#yesallwomen', 22), (u'#equality', 20), (u'#benghazi', 17), (u'#bible', 16), (u'#ccot', 15), (u'#hillary', 15), (u'#uniteblue', 14), (u'#freethinker', 14), (u'#alllivesmatter', 14), (u'#stophillary2016', 13), (u'#feminists', 13), (u'#climate', 13), (u'#blacklivesmatter', 13), (u'#christian', 12), (u'#womensrights', 12), (u'#mission', 12), (u'#baltimoreriots', 12), (u'#tip', 12), (u'#catholic', 12), (u'#whyimnotvotingforhillary', 12)]
[(u"don't", 166), (u'...', 156), (u"it's", 137), (u"i'm", 120), (u"can't", 76), (u'2', 52), (u"you're", 46), (u"doesn't", 44), (u'..', 41), (u"women's", 29), (u"that's", 28), (u"she's", 28), (u"they're", 27), (u"let's", 27), (u"isn't", 26), (u"what's", 24

In [ ]:
from nltk import TweetTokenizer

tknz = TweetTokenizer(strip_handles=True)
tknz.tokenize("greattttt amazingggggg")